In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', 500)
import arviz as az
import matplotlib.pyplot as plt

In [ ]:
classify = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/data_flipped.csv')

In [ ]:
## EDA looking at how to classify rush types
play1 = classify[classify['uniqueplayId'] == 2021091902389]
play1 = play1[play1['pff_positionLinedUp'].isin(['LT', 'RE'])]
play_data = play1[['team','new_x','new_y','o','frameId']]